## 1단계: TensorFlow의 기본 뼈대 이해하기

- 하위 주제 1: 텐서플로우란? (핵심 원리: 그래프와 이식성)
- 하위 주제 2: 텐서(Tensor)와 변수(Variable) (데이터의 기본 단위)
- 하위 주제 3: 자동 미분과 GradientTape (학습의 엔진)

## 2단계: 모델에 데이터 공급하기 (tf.data)

- 하위 주제 1: 효율적인 데이터 파이프라인의 필요성
- 하위 주제 2: tf.data.Dataset으로 데이터 불러오고 가공하기
- 하위 주제 3: 성능 최적화의 비밀: prefetch

## 3단계: 모델을 위한 재료 준비 (Feature Engineering)

- 하위 주제 1: 특성(Feature)이란? (수치형 vs 범주형)
- 하위 주제 2: 특성 열(Feature Column)의 마법 (원-핫 인코딩 & 임베딩)

## 1단계: TensorFlow의 기본 뼈대 이해하기


### 1-1 TenserFlow?
방향성 비순환 그래프(DAG, Directed Acyclic Graph)
- Node: Operaion 에 해당
- Edge: 데이터의 흐름 - Node 를 잇는 선

**텐서(데이터)**가 그래프를 따라 흐르면서(Flow) 연산되는것

파이썬으로 설계도를 생성하면 스마트폰, 웹브라우저, ... 다른 기기에서 Runtime이 실행하기 때문에 이식성(Portability)이 좋다


### 활동: 미니 레시피 분석하기
TensorFlow로 `(3 + 5) * 2` 이라는 간단한 계산을 한다고 가정. 

1. 이 레시피에서 '재료를 손질하고 섞는' 것과 같은 실제 행동 단계(연산)에 해당하는 노드(Node)는 무엇일까요?
2. 그 행동 단계들 사이를 흘러 다니는 '중간 재료(데이터)'에 해당하는 에지(Edge)는 무엇일까요?
3. 이 레시피에 따라 요리를 마쳤을 때 나오는 최종 결과물, 16이라는 숫자는 TensorFlow에서 부르는 용어로 무엇일까요?

- 노드 (Node): 덧셈, 곱셈 같은 연산.
- 에지 (Edge): 연산과 연산을 잇는 데이터의 흐름/경로.
- 텐서 (Tensor): 그 경로를 따라 흐르는 모든 데이터 (숫자, 벡터, 행렬 등).

---


### 1-2 Tensor와 Variable

Tensor?
데이터의 N차원 배열

- Rank 0 Tensor: scalar - 단일 데이터
- Rank 1 Tensor: Vector - 1차원 배열 Ex) - [1, 2, 3]
- Rank 2 Tensor: matrix - 2차원 배열 Ex) - [[1, 2, 3], [4, 5, 6]]
- Rank 3 Tensor: - 2차원 배열의 집합
- Rank N Tensor: rank n - 1 텐서의 집합

이전 텐서가 쌓여 이후 텐서를 구성한다.

수학에서의 1, 2, 3`차원` 과는 다르게, `rank tensor`는 데이터에 접근하기 위한 인덱스의 갯수로 이해를 해야한다. 

- 벡터 -> data[i]
- 행렬 -> data[i][j] -> 인덱스 2개
- 3차원 텐서 -> data[i][j][k] -> 인덱스 3개

#### 텐서의 종류

1. Constant - 상수
- tf.constant 로 생성. 이후 값은 불변

2. Variable - 변수
- tf.variable로 생성. 값을 변경 가능
- **가중치(weights)**를 저장할때 사용





In [1]:
import tensorflow as tf

# 1. 상수 (Constant): 한 번 정해지면 바꿀 수 없어요.
my_constant = tf.constant(10)
print(f"상수 텐서: {my_constant}")

# 2. 변수 (Variable): 값을 바꿀 수 있어요.
my_variable = tf.Variable(100.0)
print(f"변수 텐서 (업데이트 전): {my_variable.numpy()}")

# 변수의 값을 10만큼 더해서 업데이트합니다. (학습 과정과 유사)
my_variable.assign_add(10.0)
print(f"변수 텐서 (업데이트 후): {my_variable.numpy()}")

상수 텐서: 10
변수 텐서 (업데이트 전): 100.0
변수 텐서 (업데이트 후): 110.0


---

## 1-3 자동 미분과 GradientTape
모델의 예측이 정답과 떨어진 정도를 나타내는 **손실(Loss)** 을 최소화 하는 방향으로 가중치를 조정해야 한다.

**Loss** 가 가장 기울어 져 있는 **경사(Gradient)** 를 찾고, 가중치를 조정하는 과정을 **경사하강법(Gradient Descent)** 이라 한다.

**Loss** 가 기울어진 정도는 Loss함수를 미분하여 구할 수 있지만, 모든 가중치에 대해 손으로 계산하는 것은 불가능

`tf.GradientTape` 으로 효율적이고 쉽게 계산 가능하다.

- 녹화 시작: with tf.GradientTape() as tape: 라는 코드로 녹화기를 킨다.
- 연산 녹화: 녹화기가 켜진 동안, 모델이 예측값을 만들고 손실(loss)을 계산하는 모든 과정을 테이프에 차곡차곡 기록한다. 
- 테이프 되감기 (미분): 녹화가 끝나면, tape.gradient(loss, weights) 라는 명령으로 테이프를 거꾸로 되감는다. 이 과정에서 TensorFlow는 **연쇄 법칙(Chain Rule)**을  이용해 손실 값에 영향을 미친 모든 가중치의 그래디언트를 자동으로 계산

In [7]:
import tensorflow as tf

# 모델의 가중치(Variable)와 입력값, 정답을 준비합니다.
W = tf.Variable(3.0) # 현재 가중치
x = 2.0              # 입력 데이터
y_true = 10.0        # 실제 정답

# 1. 녹화 시작!
with tf.GradientTape() as tape:
  # 2. 녹화 중에 예측하고 손실을 계산합니다.
  y_pred = W * x   # 예측: 3.0 * 2.0 = 6.0
  loss = (y_pred - y_true)**2  # 손실: (6.0 - 10.0)^2 = 16.0

# 3. 녹화된 테이프를 이용해 그래디언트를 계산합니다.
# "loss(16.0)를 줄이려면 W(3.0)를 어느 방향으로 바꿔야 할까?"
grad_W = tape.gradient(loss, W)

print(f"현재 손실 값: {loss.numpy()}")
print(f"가중치 W에 대한 그래디언트: {grad_W.numpy()}")

현재 손실 값: 16.0
가중치 W에 대한 그래디언트: -16.0


## 2단계: 모델에 데이터 공급하기 (tf.data)

### 2-1: 효율적인 데이터 파이프라인의 필요성

- 엄청난 양의 데이터를 한번에 컴퓨터 메모리에 올리는 것은 불가능하다.
- GPU가 아무리 빨라도 다음 훈련에 사용될 데이터 배치가 준비되지 않으면 GPU가 논다 -> 엄청난 자원낭비

#### 해결책: 고성능 파이프라인
단순히 데이터를 가져오는 것을 넘어
- 파일 시스템에서 데이터를 읽고
- 데이터 순서를 무작위로 섞고 -> shuffle
- 이미지 크기를 바꾸거나 텍스트를 숫자로 변환하는 등의 전처리 수행
- 훈련에 알맞은 batch 사이즈로 묶음

__결과__ : GPU를 쉴새없이 굴릴 수 있다.


__`tf.data`__
1. 메모리 문제 해결: 필요한 만큼 점진적으로(progressively) 불러와서 메모리 걱정이 없음
2. 성능 문제: CPU가 데이터를 준비하는 작업과 GPU가 훈련하는 작업을 동시에 GPU의 유휴시간을 최소화





## 2단계: tf.data.Dataset으로 데이터 불러오고 가공하기

`tf.data.Dataset` -> 지능형 컨베이어 벨트: Source에서 물건을 가져와서 컨베이어를 지나는 동안 어떤 작업(Transformations)을 할지 알고 있음

__컨베이어 벨트를 조립하는 과정__
1. 데이터 소스 정의 - 시작점
- 메모리에 있는 데이터로 시작: `tf.data.Dataset.from_tensor_slices()
    - 이미 메모리에 불러온 작은 데이터에 사용(리스트, numpy)
    - `from_tensor_slices`는 데이터 덩어리를 slice내서 각 조각을 벨트 위에 올려놓는 역할
- 파일에서 바로 시작: `tf.data.TextLineDataset()` 또는 `tf.data.TFRecordDataset()`
    - `CSV` 같은 텍스트 파일이나 `TFRecord` 파일에서 데이터를 한 줄씩, 또는 한 레코드씩 읽어와 벨트 위에 올림 

2. 변환 작업 추가 - 벨트 위 작업대
- `.suffle()`: 섞기 작업대
    - 역할: 벨트를 지나가는 데이터의 순서를 마구 뒤섞는다. 모델이 데이터의 순서를 외우는 과적합(overfitting)을 방지하는 필수 작업
    - 원리: `buffer_size`라는 임시 저장 공간을 만들어, 그 안에서 데이터를 무작위로 뽑아 다음단계로 보냄
- `.map()`: 변환 작업대
    - 역할: 벨트를 지나가는 모든 데이터 각각에 대해 우리가 지정한 함수(- 전처리 함수)를 적용
    - 예시: 이미지의 크기를 특정 사이즈로 통일하거나 텍스트의 단어들을 숫자로 바꾸는 등의 작업
-  `.batch()`: 포장 작업대
    - 역할: 벨트 위를 지나가는 데이터들을 지정된 갯수 - `batch_size()`만큼 묶어 하나로 포장

In [5]:
import tensorflow as tf
import numpy as np

# 0. 재료 준비: 0 ~ 9까지의 데이터와 레이블
features = np.arange(10)
labels =  np.arange(10) * 10 

# 1. source - 메모리 데이터로 dataset 만들기
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

# 2. 섞기
dataset = dataset.shuffle(buffer_size=10)

# 3. 변환 - feature에 2 곱하기
def preprocess(feature, label):
    return feature * 2, label

dataset = dataset.batch(5)

for batch_data in dataset:
    print("하나의 배치: ", batch_data)


하나의 배치:  (<tf.Tensor: shape=(5,), dtype=int64, numpy=array([4, 7, 6, 9, 1])>, <tf.Tensor: shape=(5,), dtype=int64, numpy=array([40, 70, 60, 90, 10])>)
하나의 배치:  (<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 3, 8, 2, 5])>, <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 0, 30, 80, 20, 50])>)


2025-06-18 01:06:26.915995: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### 2-3: 성능 최적화의 비밀: prefetch
__GPU__ 의 속도가 매우 빨라 컨베이어 벨트가 따라가지 못하는 상황이 있을 수 있다.

`prefetch`는 이러한 상황을 막기 위해 GPU의 훈련이 끝나기 전까지 다음 훈련 데이터를 준비한다.

**`prefetch`** 를 단순히 파이프라인 마지막에 추가하여 사용 가능!

```python
# ...
dataset = dataset.batch(5)

# 파이프라인 마지막에 추가
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

print("최종 파이프라인", dataset)

```

## 3단계: 모델을 위한 재료 준비 (Feature Engineering)

### 3-1: 특성(Feature)이란? (수치형 vs 범주형)

지금까지 해왔던 과정은, 그릇(Tensor) 에 담을 데이터를 컨베이어 벨트에 준비해온 일이다.

#### Tensor 에 담을 데이터를 손질하는 방법
- 특성(Feature) - 예측의 기반이 되는 힌트
    1. 수치형 데이터 - Numerical Data
        - 그 자체로 숫자이며 크기를 비교할 수 있는 데이터
        - EX) 나이, 온도, 가격, 면적
        - 약간의 처리만 거치면(값의 범위를 0~1로 맞추는 정규화...etc) 바로 모델에 사용 가능
    2. 범주형 데이터 - Categorical Data
        - 정해진 몇가지 범주에 속하는 데이터, 대소관계 없음
        - EX) 도시, 혈액형, 부동산 유형
        - 모델은 위의 예시의 의미를 이해하지 못해 숫자형태로 바꿔주는 변환 필요

**범주형 데이터**
- 성별을 남성 = 0 , 여성 = 1 로 바꾸면? 괜찮아 보임
- 하지만 서울 = 0, 부산 = 1, 대구 = 2 로 바꾸면, 모델이 대소관계를 인식할 수도 있다

이것을 처리해 주는것이 TensorFlow의 Feature Column 이다. -> 원 핫 인코딩, 임베딩


### 3-2: 특성 열(Feature Column)의 마법 (원-핫 인코딩 & 임베딩)
모델과 데이터 사이를 이어주는 만능 번역기

1. 원핫 인코딩: 작은 사전을 위한 번역법
    - When? : 성별, 요일, 혈액형 과 같이 적은 수의 범주형
    - How? : 각 범주에 해당하는 자리만 1(Hot), 나머지는 모두 0으로 끄는 방식
        - 어휘 목록: ['남성', '여성']
            - '남성' -> [1, 0]
            - '여성' -> [0, 1]
        - 어휘 목록: ['서울', '부산', '인천']
            - '서울' -> [1, 0, 0]
            - '부산' -> [0, 1, 0]
            - '인천' -> [0, 0, 1]
    - 문제: 만약 수백만 개의 범주일 경우, [0, 0, ..., 1] 과 같이 대부분이 0으로 채워진 비효율 적인 __'희소 벡터(Sparse Vector)'__ 가 만들어진다. -> 메모리 낭비

2. 임베딩(Embedding): 거대한 사전을 위한 번역법
    - When? : 영화 제목, 상품 종류 와 같이 범주의 개수가 아주 많을 때
    - How? : 각 범주를 의미를 함축한 저차원의 __'밀집 벡터(Dense Vector)'__ 로 표현
    - 원 핫 인코딩이 각 단어에 번호를 부여하는 방식이라면 임베딩은 각 단어에 __좌표__ 를 찾아주는 것
        - '왕': [0.9, 0.8, 0.1]
        - '여왕': [0.8, 0.9, 0.1]
        - '사과': [0.1, 0.2, 0.9]
    - 모델이 훈련 과정에서 스스로 단어들의 관계를 학습하며 최적의 좌표값을 찾아낸다. -> '왕' 과 '여왕'은 가깝게, '사과'는 멀게
    - 장점: 메모리 효율적, 단어 사이의 유사성까지 학습



